<a href="https://colab.research.google.com/github/marianelia/tmp/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Solution_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лабораторная работа по курсу "Искусственный интеллект"
## Многослойный персептрон

| Студент | Лагуткина |
|---------|--------|
| Группа  | 6      |

Вариант: 1

Для начала, скачаем датасет MNIST. Используйте `wget` или `curl`, либо скачайте вручную [по ссылке](https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/Data/MNIST/mnist.pkl.gz).

## Нейронная сеть 
### Установка библиотек и настройка среды


In [1]:
import torch
import numpy as np
import pandas
import torchvision.utils
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
print(torch.__version__)

1.11.0+cu113


### Устанавливаем вычислительное устройство

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.cuda.device_count())
def print_device():
  print('Doing computations on '+device)

0


## Нейросеть


In [20]:
class Perceptron_solution_pytorch(torch.nn.Module):
    def __init__(self,learning_rate=0.1,epochs=10, layers=[], trans_f=torch.nn.Tanh(), loss_f=torch.nn.functional.binary_cross_entropy_with_logits):
        super().__init__()
        self.learning_rate=learning_rate
        self.layers=layers
        self.transfer=trans_f
        self.epochs=epochs
        self.loss_function=loss_f
        self.level=0

    #создание сети и настройка вывода
    def create(self, X,Y):
        self.level=0
        layers_tmp = self.layers.copy()
        layers_tmp.insert(0, len(X[0]))
        layers_tmp.append(len(np.unique(Y)))
        self.classes_count = layers_tmp[-1]
        module_layers = []
        self.print_format("Network Training:")
        self.level += 1
        self.print_format("In | {}".format(layers_tmp[0]))
        self.print_format("Out | {}".format(layers_tmp[-1]))
        self.print_format("Hidden layers    | {}".format(len(layers_tmp)-2))
        self.print_format("The breadth of hidden layers    | {}".format(self.layers))
        self.level -= 1
        for i in range(len(layers_tmp)-2):
            #собираем слой
            module_layers.append(torch.nn.Linear(layers_tmp[i], layers_tmp[i+1]))
            module_layers.append(self.transfer) 
        module_layers.append(torch.nn.Linear(layers_tmp[-2], layers_tmp[-1]))
        self.net = torch.nn.Sequential(*module_layers)
        return self      

    #создание и обучение сети    
    def training(self, X, Y):
        self.create(X,Y)
        self.level = 0
        self.print_format("Network Training:")
        self.level += 1

        x_value = torch.tensor(np.array(X).astype(np.float32))
        y_value = []
        for y in Y:
            val = np.zeros(self.classes_count)
            for i in range(10):
                val[i] = 1 if i == y else 0
            y_value.append(val.copy().astype(np.float32))
        dataset = torch.utils.data.TensorDataset(x_value,torch.tensor(y_value, dtype=torch.float32))
        #разделим входные данные на мини-батчи размером 16 
        dataloader = torch.utils.data.DataLoader(dataset,batch_size=16)
        #оптимизация
        optim = torch.optim.Adam(self.net.parameters(),lr=self.learning_rate)
        for epoch in range(1, self.epochs+1):
            for (x,y) in dataloader:
                input = self.net(x)
                loss = self.loss_function(input,y) #target=y
                optim.zero_grad()
                loss.backward()
                optim.step()
            acc = self.accuracy(X,Y)
            self.print_format("Epoch {} of {}: loss : {}, accuracy : {}".
                              format(epoch, self.epochs, loss, acc))
        return self

    def print_format(self, msg, level=-1):
        if level == -1:
            level = self.level
        print("  "*level + msg)  

    def accuracy_plot(self, acc):
        plt.plot(self.level, acc, label='Accuracy')
        plt.legend()
        plt.show()

    #точность        
    def accuracy(self, X, Y):
        x_value = torch.tensor(np.array(X).astype(np.float32))
        answer = self.net(x_value)
        results = [np.argmax(v.detach().numpy()) for v in self.transfer(answer)]
        total_success = 0
        for res, y in zip(results, Y):
            if res == y:
                total_success += 1
        acc = total_success / len(Y)
        plt.plot(self.level, acc, label='Accuracy')
        plt.legend()
        plt.show()
        return acc

    def confusion_matrix(self, X, Y):
        x_value = torch.tensor(np.array(X).astype(np.float32))
        answer = self.net(x_value)
        results = [np.argmax(v.detach().numpy()) for v in self.transfer(answer)]
        matrix = np.zeros((self.classes_count, self.classes_count), dtype=np.int32)
        for res, y in zip(results, Y):
            matrix[res][y] += 1
        return matrix




## Загрузка датасетов и подготовка данных

### Подготовительные функции

In [15]:
def convert_values(df):
    return np.array(df).astype(np.float32)/255

def count_of_keys(data):
    unique = np.unique(data)
    count = np.zeros(len(unique), dtype=np.int32)
    for v in data:
        count[v] += 1
    return count

def print_data(labels, features, string :str):
    print("{}".format(string))
    print("Number of elements in the dataset : {}".format(len(features[0])))
    print("The number of attributes in the element : {}".format(len(labels)))
    print("Minimum attribute value  : {}".format(np.min(features)))
    print("Maximum attribute value  : {}".format(np.max(features)))
    print()
            
def analize(labels, features, string :str):
    print_data(labels, features, string)

#рисовалка графиков аккюраси

### MNIST

In [16]:
mnist_train = datasets.MNIST(root = './data/', train = True, transform = transforms.ToTensor(), download=True)
mnist_train_labels = mnist_train.train_labels;
mnist_train_data_raw = mnist_train.train_data;

print(mnist_train_data_raw.shape)
mnist_train_data = convert_values(mnist_train_data_raw)
mnist_train_data_reshape = mnist_train_data.reshape(60000, -1)

analize(mnist_train_labels, mnist_train_data_reshape, "Training sample from MNIST")

mnist_test = datasets.MNIST(root = './data/', train = False,transform = transforms.ToTensor(), download=True)
mnist_test_lebels = mnist_test.train_labels;
mnist_test_data_raw = mnist_test.train_data;

print(mnist_test_data_raw.shape)
mnist_test_data = convert_values(mnist_test_data_raw)
mnist_test_data_reshape = mnist_test_data.reshape(10000, -1)

analize(mnist_test_lebels, mnist_test_data_reshape, "Testing sample from MNIST")

torch.Size([60000, 28, 28])
Training sample from MNIST
Number of elements in the dataset : 784
The number of attributes in the element : 60000
Minimum attribute value  : 0.0
Maximum attribute value  : 1.0

torch.Size([10000, 28, 28])
Testing sample from MNIST
Number of elements in the dataset : 784
The number of attributes in the element : 10000
Minimum attribute value  : 0.0
Maximum attribute value  : 1.0



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


### FasionMNIST

In [17]:
fashion_mnist_train = datasets.FashionMNIST(root = './fdata/', train = True, transform = transforms.ToTensor(), download=True)
fashion_mnist_train_labels = fashion_mnist_train.train_labels;
fashion_mnist_train_data_raw = fashion_mnist_train.train_data;

print(fashion_mnist_train_data_raw.shape)
fashion_mnist_train_data = convert_values(fashion_mnist_train_data_raw)
fashion_mnist_train_data_reshape = fashion_mnist_train_data.reshape(60000, -1)

analize(fashion_mnist_train_labels, fashion_mnist_train_data_reshape, "Training sample from Fashion MNIST")

fashion_mnist_test = datasets.MNIST(root = './fdata/', train = False,transform = transforms.ToTensor(), download=True)
fashion_mnist_test_labels = fashion_mnist_test.train_labels;
fashion_mnist_test_y_raw = fashion_mnist_test.train_data;

print(fashion_mnist_test_y_raw.shape)
fashion_mnist_test_y = convert_values(fashion_mnist_test_y_raw)
fashion_mnist_test_y_reshape = fashion_mnist_test_y.reshape(10000, -1)

analize(fashion_mnist_test_labels, fashion_mnist_test_y_reshape, "Testing sample from Fashion MNIST")

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


torch.Size([60000, 28, 28])
Training sample from Fashion MNIST
Number of elements in the dataset : 784
The number of attributes in the element : 60000
Minimum attribute value  : 0.0
Maximum attribute value  : 1.0

torch.Size([10000, 28, 28])
Testing sample from Fashion MNIST
Number of elements in the dataset : 784
The number of attributes in the element : 10000
Minimum attribute value  : 0.0
Maximum attribute value  : 1.0



### CIFAR-10

Датасет CIFAR-10 состоит из цветных изображений размером 32х32. Для картинок есть 10 категорий.

#### Загрузка датасета

In [18]:
train_size = 50000
test_size = 10000

cfar_train = datasets.CIFAR10(root = './cdata/', train = True, transform = transforms.ToTensor(), download=True)
cfar_train_list =  torch.utils.data.DataLoader(cfar_train, batch_size=train_size, shuffle=True, num_workers=2)

cfar_test = datasets.CIFAR10(root = './cdata/', train = False,transform = transforms.ToTensor(), download=True)
cfar_test_list =  torch.utils.data.DataLoader(cfar_test, batch_size=test_size, shuffle=True, num_workers=2)

train_dataiter = iter(cfar_train_list)
cfar_train_images, cfar_train_labels = train_dataiter.next()
cfar_train_images_reshape = cfar_train_images.reshape(50000,-1)

print(cfar_train_images_reshape.shape)
print(cfar_train_labels.shape)

dataiter = iter(cfar_test_list)
cfar_test_images, cfar_test_labels = dataiter.next()
cfar_test_images_reshape = cfar_test_images.reshape(10000,-1)

print(cfar_test_images_reshape.shape)
print(cfar_test_labels.shape)

cfar_train_images_reshape_array= np.array(cfar_train_images_reshape)
cfar_test_images_reshape_array= np.array(cfar_test_images_reshape)

analize(cfar_train_labels, cfar_train_images_reshape_array, "ОTraining sample from CIFAR-10")

analize(cfar_test_labels, cfar_test_images_reshape_array, "Testing sample from CIFAR-10")

HTTPError: ignored

## Обучение и тестирование сети  

`In` - количество входных нейронов

`Out` - количество выходных нейронов

`Hidden layers` - число скрытых слоев

`The breadth of hidden layers` - широта скрытых слоев

In [ ]:
def print_accuracy(accuracy):
    print("Network accuracy : {}".format(accuracy))

#### MNIST

Передаточная функция: 

Количество слоев:


In [24]:
null_layer = Perceptron_solution_pytorch(layers=[])
null_layer.training(mnist_train_data_reshape, mnist_train_labels)
print_accuracy(null_layer.accuracy(mnist_test_lebels,mnist_test_data_reshape))

TypeError: ignored

Передаточная функция: 

Количество слоев:

Передаточная функция: 

Количество слоев:

Передаточная функция: 

Количество слоев: